In [1]:
#Imports
import os
import re
from collections import Counter
from pprint import pprint
import json

In [2]:
from scripts import OpenAI_lib as ol
import time

client = ol.get_client_local()

In [ ]:
with open("data/o")

In [ ]:
from openai import OpenAI
from pydantic import BaseModel
from typing import Dict
from openai.types.responses import Response
from openai.lib._parsing._responses import parse_response

#Define wished output structure and method for prompting GPT
class GeneratedTexts(BaseModel):
    alku: str
    teksti: str

file_response = client.files.content()

completions = [
    parse_response(
        response=Response\
            .model_validate(json.loads(line)['response']['body']),
        text_format=GeneratedTexts,
        input_tools=[]
    )\
    .output_parsed\
    for line in file_response.read().splitlines() if line.strip()
]

pprint(completions)

with open("data/GPTRegens/news-fi-2019.jsonl_regeneration_test_1.jsonl", 'r') as reader:
    for l in reader:
        if len(l) > 0:
            tester = json.loads(l.strip())

pprint(tester)

In [129]:
texts = []
metadata = []
for fn in os.listdir("data/TDT"):
    with open("data/TDT/"+fn, 'r') as reader:
        for l in reader:
            if l.find("# sent_id = ") != -1:
                metadata.append(l[12:-1])
                continue
            if l.find(" text = ") != -1:
                texts.append(l[8:-1])

In [130]:
assert len(texts) == len(metadata)

In [131]:
print(len(texts))

15136


In [132]:
abbrs = []
for n in metadata:
    val = int(re.search(r'\d+', n).start())
    abbrs.append(n[:val])

In [133]:
full_docs = {x[:x.find('.')]:[] for x in metadata if x[0] != 'h'}

In [134]:
acrs = {
    'wn' : 'wikinews-artikkeli',
    'f' : 'kaunokirjallinnen teksti',
    'fA' : 'kaunokirjallinnen teksti',
    'fB' : 'kaunokirjallinnen teksti',
    'fC' : 'kaunokirjallinnen teksti',
    'fD' : 'kaunokirjallinnen teksti',
    'fE' : 'kaunokirjallinnen teksti',
    'fF' : 'kaunokirjallinnen teksti',
    'b' : 'blogiteksti',
    'u' : 'yliopisto uutinen',
    'j' : 'lakiteksti',
    'e' : 'parlamenttipuhe',
    't' : 'talousuutinen',
    'w' : 'wikipedia-artikkeli',
    's' : 'opiskelijalehti-artikkeli',
    'h' : 'esimerkki'
}
for a in set(abbrs):
    acrs[a]

ool = {
    'wn' : 'otsikko',
    'f' : 'otsikko',
    'fA' : 'otsikko',
    'fB' : 'otsikko',
    'fC' : 'otsikko',
    'fD' : 'otsikko',
    'fE' : 'otsikko',
    'fF' : 'otsikko',
    'b' : 'otsikko',
    'u' : 'otsikko',
    'j' : 'otsikko',
    'e' : 'ensimmäinen lause',
    't' : 'otsikko',
    'w' : 'enismmäinen lause',
    's' : 'otsikko',
    'h' : 'esimerkki'
}

In [135]:
for i,n in enumerate(metadata):
    if n[0] != 'h':
        full_docs[n[:n.find('.')]] = full_docs[n[:n.find('.')]] + [texts[i]]

full_docs_list = []
for f in full_docs:
    sents = full_docs[f]
    full_docs_list.append({'id':f, 'register':acrs[f[:int(re.search(r'\d+', f).start())]], 'first_line_type':ool[f[:int(re.search(r'\d+', f).start())]], 'first_line':sents[0], 'text':' '.join(sents[1:]), 'text_sent_amount':len(sents)-1})

In [136]:
print(full_docs_list[0])

{'id': 'b104', 'register': 'blogiteksti', 'first_line_type': 'otsikko', 'first_line': ' Taas teatteriin', 'text': ' Tänäänkin pitäisi mennä teatteriin.  Varasin pupulle ja minulle sekä sille sisarentyttärelleni, joka pääsi Turkuun lakia lukemaan, liput kaupunginteatterin Laulavat sadepisarat -musikaaliin.  Sen jälkeen voisi mennä vaikkapa Suxessiin oluelle.  Muita matkalla sinne?', 'text_sent_amount': 4}


In [137]:
print(len(full_docs_list))

674


In [138]:
with open("TDT_doc_data.jsonl", 'w') as writer:
    for f in full_docs_list:
        writer.write(json.dumps(f)+'\n')